In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
"""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
"""

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n        break\n"

In [2]:
import soundfile
import numpy as np
import librosa
import glob
import os
from sklearn.model_selection import train_test_split
import sys
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix

In [3]:
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [5]:
# all emotions present in the dataset
int2emotion = {
    "01": "surprise",
    "02": "fear",
    "03": "neutral",
    "04": "sadness",
    "05": "sarcastic",
    "06": "disgust",
    "07": "happy",
    "08": "anger"
}

# allow only these emotions
AVAILABLE_EMOTIONS = {
    "fear",
    "neutral",
    "sadness",
    "happy",
    "anger",
    "sarcastic"
}

**Feature Extraction**

In [4]:
def extract_feature(file_name, **kwargs):
    FRAME_LENGTH = 1024
    HOP_LENGTH = 512
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    rolloff = kwargs.get("rolloff")
    rms = kwargs.get("rms")
    harmonic = kwargs.get("harmonic")
    band = kwargs.get("band")
    centroid = kwargs.get("centroid")
    zcr = kwargs.get("zcr")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr= sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
        if rolloff:
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate,roll_percent=0.85).T,axis=0)
            result = np.hstack((result, rolloff))
        if rms:
            rms = np.std(librosa.feature.rms(y=X, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH).T,axis=0)
            result = np.hstack((result, rms))
        if harmonic:
            y_harmonic, y_percussive = librosa.effects.hpss(y=X)
            harmonic = np.mean(y_harmonic.T,axis=0)
            result = np.hstack((result,harmonic))
        if band:
            band=np.mean(librosa.feature.spectral_bandwidth(y=X,sr=sample_rate,n_fft=FRAME_LENGTH,hop_length=HOP_LENGTH).T,axis=0)
            result = np.hstack((result,band))
        if centroid:
            centroid=np.mean(librosa.feature.spectral_centroid(y=X,sr=sampling_rate,n_fft=FRAME_LENGTH,hop_length=HOP_LENGTH).T,axis=0)
            result = np.hstack((result,centroid))
        if zcr:
            zcr = np.mean(librosa.feature.zero_crossing_rate(y=X,frame_length=FRAME_LENGTH,hop_length=HOP_LENGTH).T,axis=0)
            result = np.hstack((result,zcr))
    return result

In [22]:
def load_data(test_size=0.2):
    X,y = [],[]
    try :
        for file in glob.glob('../input/main-dataset/IITKGP_SEHSC/**//*.wav',recursive=True):
            # get the base name of the audio file
            basename = os.path.basename(file)
            print(basename)
            # get the emotion label
            emotion = (basename.split(".")[2]).split("-")[0]
            # we allow only AVAILABLE_EMOTIONS we set
            if emotion not in AVAILABLE_EMOTIONS:
                continue
          # extract speech features
            features = extract_feature(file,mfcc=True)
          # add to data
            X.append(features)
            y.append(emotion)
            
    except :
         pass
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, random_state=42)

In [ ]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])
# best model, determined by a grid search
model_params = {
    'alpha': 0.01,
    'batch_size': 256,
    'epsilon': 1e-08, 
    'hidden_layer_sizes': (300,), 
    'learning_rate': 'adaptive', 
    'max_iter': 500, 
}
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

# now we save the model
# make result directory if doesn't exist yet
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

In [ ]:
model = MLPClassifier(**model_params)

# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

In [23]:
X_train, X_test, y_train, y_test = load_data(test_size=0.25)
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using utils.extract_features() method
print("[+] Number of features:", X_train.shape[1])

5.10.surprise-02.wav
5.10.surprise-08.wav
5.10.surprise-07.wav
5.10.surprise-13.wav
5.10.surprise-12.wav
5.10.surprise-10.wav
5.10.surprise-04.wav
5.10.surprise-14.wav
5.10.surprise-06.wav
5.10.surprise-05.wav
5.10.surprise-03.wav
5.10.surprise-09.wav
5.10.surprise-15.wav
5.10.surprise-01.wav
5.10.surprise-11.wav
5.10.fear-10.wav
5.10.fear-04.wav
5.10.fear-08.wav
5.10.fear-07.wav
5.10.fear-01.wav
5.10.fear-06.wav
5.10.fear-13.wav
5.10.fear-14.wav
5.10.fear-12.wav
5.10.fear-09.wav
5.10.fear-15.wav
5.10.fear-11.wav
5.10.fear-02.wav
5.10.fear-05.wav
5.10.fear-03.wav
5.10.neutral-07.wav
5.10.neutral-08.wav
5.10.neutral-09.wav
5.10.neutral-06.wav
5.10.neutral-13.wav
5.10.neutral-04.wav
5.10.neutral-12.wav
5.10.neutral-14.wav
5.10.neutral-05.wav
5.10.neutral-01.wav
5.10.neutral-15.wav
5.10.neutral-03.wav
5.10.neutral-10.wav
5.10.neutral-11.wav
5.10.neutral-02.wav
5.10.sadness-02.wav
5.10.sadness-07.wav
5.10.sadness-06.wav
5.10.sadness-13.wav
5.10.sadness-12.wav
5.10.sadness-03.wav
5.10.sadne

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test)

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

In [ ]:
dtree_model = DecisionTreeClassifier(max_depth = 6).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=dtree_predictions))
print(classification_report(y_test,dtree_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, dtree_predictions) )

In [ ]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 

print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

In [ ]:
filename = '../input/new-testing/Recording.wav'

In [ ]:
fileofmeangry = '/kaggle/input/new-testing/Recording.wav'
anspredictionmade = []
new_featureused = extract_feature(fileofmeangry, mfcc=True, chroma=True, mel=True)
anspredictionmade.append(new_featureused)
anspredictionmade = np.array(anspredictionmade)

In [20]:
def extract_mfcc(filename):
    yoyo,srike = librosa.load(filename,duration=3,offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=yoyo,sr=srike,n_mfcc=40).T,axis=0)
    return mfcc

In [30]:
from sklearn.ensemble import RandomForestClassifier
  
 # create regressor object 
classifier = RandomForestClassifier(n_estimators = 200, random_state = 0) 
  
# fit the regressor with x and y data 
classifier.fit(X_train, y_train)
lengthoftestset = len(X_test)
#rand_no = random.randint(0,lengthoftestset-1)
new_data = extract_mfcc('/kaggle/input/new-testing/Recording.wav')
#print(X_test)
#print(rand_no)
reshaped_data = new_data.reshape(1,-1)
c_p = classifier.predict(reshaped_data) 
emotion_iden = c_p[0]
print(emotion_iden)

"""
print(accuracy_score(y_true=y_test,y_pred=c_p))
print(classification_report(y_test,c_p)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,c_p) )
"""

neutral


'\nprint(accuracy_score(y_true=y_test,y_pred=c_p))\nprint(classification_report(y_test,c_p)) \n# creating a confusion matrix \nprint(confusion_matrix(y_test,c_p) )\n'

In [9]:
import random

In [11]:
directory_angry = '/kaggle/input/turkish-music-emotion-dataset/angry'
directory_happy = '/kaggle/input/turkish-music-emotion-dataset/happy'
directory_relax = '/kaggle/input/turkish-music-emotion-dataset/relax'
directory_sad = '/kaggle/input/turkish-music-emotion-dataset/sad'

In [25]:
def play_music(emotion_identified):
    if(emotion_identified == 'anger'):
        file = random.choice(os.listdir(directory_angry))
        print(file)
        full_name = '/kaggle/input/turkish-music-emotion-dataset/angry/' + file
        y, sr = librosa.load(full_name)
        return y
    elif(emotion_identified == 'happy'):
        file = random.choice(os.listdir(directory_happy))
        print(file)
        full_name = '/kaggle/input/turkish-music-emotion-dataset/happy/' + file
        y, sr = librosa.load(full_name)
        return y
    elif(emotion_identified == 'sad'):
        file = random.choice(os.listdir(directory_sad))
        print(file)
        full_name = '/kaggle/input/turkish-music-emotion-dataset/sad/' + file
        y, sr = librosa.load(full_name)
        return y
    else:
        file = random.choice(os.listdir(directory_relax))
        print(file)
        full_name = '/kaggle/input/turkish-music-emotion-dataset/relax/' + file
        y, sr = librosa.load(full_name)
        return y

In [26]:
values = play_music(emotion_iden)

bulent_ortacgil_yuzunu_dokme_kucuk_kiz.mp3


In [27]:
Audio(data = values, rate = 22050)

In [15]:
from IPython.display import Audio

In [ ]:
data, sampling_rate = librosa.load('../input/new-testing/Recording.wav')
data = np.array(data)

In [ ]:
"""ans =[]
new_feature ,labels  = extract_feature(filename, mfcc=True, chroma=False, mel=False,contrast=False,tonnetz=False)
np.expand_dims(new_feature,-1)
ans.append(new_feature)
ans = np.array(ans)
# data.shape

classifier.predict([ans])
"""

In [31]:
from sklearn.ensemble import AdaBoostClassifier
AdaModel = AdaBoostClassifier(n_estimators=300,learning_rate=1.0)
model = AdaModel.fit(X_train,y_train)
c_p1 = model.predict(reshaped_data) 
emotion_ident = c_p1[0]
print(emotion_ident)

"""
cpre = model.predict(X_test)
print(accuracy_score(y_true=y_test,y_pred=cpre))
print(classification_report(y_test,cpre)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,cpre))
"""

happy


'\ncpre = model.predict(X_test)\nprint(accuracy_score(y_true=y_test,y_pred=cpre))\nprint(classification_report(y_test,cpre)) \n# creating a confusion matrix \nprint(confusion_matrix(y_test,cpre))\n'

In [32]:
from sklearn.linear_model import LogisticRegression
myLogRegmodel = LogisticRegression(max_iter=100)
model2 = myLogRegmodel.fit(X_train,y_train)
c_p2 = model2.predict(reshaped_data) 
emotion_identi = c_p2[0]
print(emotion_identi)
"""
prediction = model2.predict(X_test)
print(accuracy_score(y_true=y_test,y_pred=prediction))
print(classification_report(y_test,prediction)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,prediction) )
"""

neutral


'\nprediction = model2.predict(X_test)\nprint(accuracy_score(y_true=y_test,y_pred=prediction))\nprint(classification_report(y_test,prediction)) \n# creating a confusion matrix \nprint(confusion_matrix(y_test,prediction) )\n'

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train,y_train)
knn_pre = knn.predict(X_test)
print(accuracy_score(y_true=y_test,y_pred=knn_pre))
print(classification_report(y_test,knn_pre)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,knn_pre) )

In [ ]:
from sklearn.naive_bayes import GaussianNB
naiveBayes = GaussianNB()
naiveBayes.fit(X_train,y_train)
nbpredict = naiveBayes.predict(X_test)
print(accuracy_score(y_true=y_test,y_pred=nbpredict))
print(classification_report(y_test,nbpredict)) 
# creating a confusion matrix 
print(confusion_matrix(y_test,nbpredict) )